Установка необходимых библиотек:
pip install tensorflow
pip install keras
pip install opencv-python
pip install scikit-learn

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout


In [ ]:
def load_video(path):
    cap = cv2.VideoCapture(path)
    frames = []
    while(cap.isOpened()):
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (128, 128))
        frames.append(frame)
    cap.release()
    return frames

def preprocess_data(max_frames):
    X = []
    y = []
    for path in [real_videos_path, deepfake_videos_path]:
        for video_file in os.listdir(path):
            video_path = os.path.join(path, video_file)
            frames = load_video(video_path)
            while len(frames) < max_frames:
                frames.append(np.zeros(frames[0].shape, dtype=np.uint8))
            X.append(frames[:max_frames])
            y.append(0 if path == real_videos_path else 1)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

max_frames = 100  # Максимальное количество кадров для преобразования

real_videos_path = "/content/R"
deepfake_videos_path = "/content/Deep"

X_train, X_test, y_train, y_test = preprocess_data(max_frames)

In [ ]:
model = Sequential()
model.add(Conv3D(16, kernel_size=(3, 3, 3), activation='relu', input_shape=X_train.shape[1:]))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv3d (Conv3D)             (None, 98, 126, 126, 16)  1312      
                                                                 
 max_pooling3d (MaxPooling3D  (None, 49, 63, 63, 16)   0         
 )                                                               
                                                                 
 conv3d_1 (Conv3D)           (None, 47, 61, 61, 32)    13856     
                                                                 
 max_pooling3d_1 (MaxPooling  (None, 23, 30, 30, 32)   0         
 3D)                                                             
                                                                 
 flatten (Flatten)           (None, 662400)            0         
                                                                 
 dense (Dense)               (None, 64)                4

In [4]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)


Epoch 1/10
2/2 [==============================] - 450s 242s/step - loss: 1248.0029 - accuracy: 0.5625 - val_loss: 72.1048 - val_accuracy: 0.7500
Epoch 2/10
2/2 [==============================] - 424s 219s/step - loss: 1893.2375 - accuracy: 0.4375 - val_loss: 219.0242 - val_accuracy: 0.4375
Epoch 3/10
2/2 [==============================] - 446s 248s/step - loss: 395.3583 - accuracy: 0.6406 - val_loss: 224.7690 - val_accuracy: 0.3125
Epoch 4/10
2/2 [==============================] - 421s 223s/step - loss: 134.8585 - accuracy: 0.6094 - val_loss: 48.5736 - val_accuracy: 0.5000
Epoch 5/10
2/2 [==============================] - 438s 239s/step - loss: 55.0321 - accuracy: 0.6562 - val_loss: 29.3218 - val_accuracy: 0.4375
Epoch 6/10
2/2 [==============================] - 447s 243s/step - loss: 28.2491 - accuracy: 0.6250 - val_loss: 27.0446 - val_accuracy: 0.4375
Epoch 7/10
2/2 [==============================] - 420s 221s/step - loss: 4.6119 - accuracy: 0.7188 - val_loss: 15.2809 - val_accuracy:

In [5]:
model.save('model.h5')


In [8]:
# Сохранение модели как JSON
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)


test1 - r
test2 - d
test3 - r
test4 - d
test5 - d
test6 - r


In [53]:
test_video_path = "test6.mp4"  # Путь к файлу с тестовым видео
test_frames = load_video(test_video_path)  # Загрузка и предобработка кадров
while len(test_frames) < max_frames:
    test_frames.append(np.zeros(test_frames[0].shape, dtype=np.uint8))
test_frames = np.array(test_frames[:max_frames])  # Преобразование к массиву кадров

prediction = model.predict(np.array([test_frames]))  # Предсказание
print(prediction[0])
if prediction[0] < 0.3:
    result = "Real Video"
else:
    result = "Deepfake Video"

print("The video is:", result)

1/1 [==============================] - 2s 2s/step
[2.653525e-08]
The video is: Real Video
